# Weatherdata for IPM Plateform

## 1. import weather data modules

In [20]:
from weatherdata.ipm import WeatherDataHub 

## 2. Access to weatherdatasource available on the platefom

In [21]:
ws=WeatherDataHub()

In [22]:
ws.list_resources()

{'Met Norway Locationforecast': '9-day forecasts for the entire planet. 2.5 km resolution in the Nordic-Baltic region, 9km elsewhere',
 'FMI weather forecasts': '36 hour forecasts for Finland from the Finnish Meteorological Institute',
 'Finnish Meteorological Institute measured data': 'Weather station network covering Finland. Data availability varies between the stations. The oldest station is from 1844',
 'Landbruksmeteorologisk tjeneste': 'Weather station network covering major agricultural areas of Norway. Data before 2010 are available by request. Email lmt@nibio.no',
 'MeteoBot API': "Network of privately owned weather stations. Access requires permission from each weather station's owner",
 'Fruitweb': "Network of privately owned weather stations of several types; Davis, Metos and self manufactured. Access requires permission from each weather station's owner",
 'Metos': "Network of privately owned weather stations of several types; Davis, Metos and self manufactured. Access re

## 3. Get ressource for a specific weatherdataresource 

To connect to the meteo resource, simply enter the name of the meteo resource in the get_ressource function. 

In [23]:
fmi=ws.get_ressource(name='Finnish Meteorological Institute measured data')

Once connected to the resource various functions are available. You can check 
* the available stations for the resource with station_ids function. This function This function returns a dataframe of the stations available for the resource with their names, their identifiers and the coordinates of the station (latitude, longitude)
* the weather parameters accepted by the resource  with parameters function. This function return common and optional weather parameter 
* Get Weatherdata in xarray dataset with attribute or json format (more description of this function below)

In [24]:
fmi.station_ids()

,name,id,coordinates
0,Alajärvi Möksy,101533,"[24.26, 63.09]"
1,Asikkala Pulkkilanharju,101185,"[25.52, 61.27]"
2,Enontekiö Kilpisjärvi Saana,102017,"[20.85, 69.04]"
3,Enontekiö Kilpisjärvi,102016,"[20.79, 69.05]"
4,Enontekiö Näkkälä,102019,"[23.58, 68.6]"
...,...,...,...
203,Virolahti Koivuniemi,101231,"[27.67, 60.53]"
204,Virrat Äijänneva,101310,"[23.54, 62.33]"
205,Ylitornio Meltosjärvi,101908,"[24.65, 66.53]"
206,Ylivieska airfield,101690,"[24.72, 64.05]"


In [27]:
fmi.parameters()

{'common': [1002, 3002, 2001, 4003], 'optional': None}

### Get weather data

According to weather resources differents parameters can be used:
* *for historic weatherdata*
    * parameters: list of weather parameters available for the resource selected
    * station_id: list of station id available for the resource selected
    * timeStart: Start date of the request
    * timeEnd: End date of the request
    * timezone: The timezone
    * format: 'ds' to obtain a xarray dataset or 'json' to obtain the json     
    
    
* *for forecasts weather resources*
    * latitude: list of latitude
    * longitude: list of longitude
    * altitude: list of altitude

#### Example for historic weather ressources 
* for one station_id

In [10]:
# for one station_ids
ds=fmi.data(parameters=[1002,3002],
            station_id=[101533],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timezone='UTC',
            format='ds')
ds

start connecting to station 101533
data is ok


* for several station_ids

In [29]:
ds=fmi.data(parameters=[1002,3002],
            station_id=[101533,101185],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timezone='UTC',
            format='ds')
ds

start connecting to station 101533
data is ok
start connecting to station 101185
data is ok


<xarray.Dataset>
Dimensions:   (alt: 1, lat: 2, location: 2, lon: 2, time: 505)
Coordinates:
  * lat       (lat) float64 61.27 63.09
  * location  (location) int64 101185 101533
  * time      (time) datetime64[ns] 2020-06-12 ... 2020-07-03
  * lon       (lon) float64 24.26 25.52
  * alt       (alt) float64 0.0
Data variables:
    1002      (lon, lat, time, location) float64 nan nan nan nan ... nan nan nan
    3002      (lon, lat, time, location) float64 nan nan nan nan ... nan nan nan
Attributes:
    weatherRessource:  Finnish Meteorological Institute measured data
    weatherStationId:  [101533, 101185]
    longitude:         [24.26084, 25.52021]
    latitude:          [61.26521, 63.08898]
    timeStart:         2020-06-12T00:00:00.000000000
    timeEnd:           2020-07-03T00:00:00.000000000
    parameters:        ['1002', '3002']

#### For forecast weather resources
* example for one latitude, longitude, altitude

In [30]:
norway=ws.get_ressource(name='Met Norway Locationforecast')

In [34]:
ds=norway.data(latitude=[67.2828], longitude=[14.3711], altitude=[70],format='ds')
ds

<xarray.Dataset>
Dimensions:   (alt: 1, lat: 1, location: 1, lon: 1, time: 226)
Coordinates:
  * time      (time) datetime64[ns] 2021-05-17T09:00:00 ... 2021-05-26T18:00:00
  * location  (location) <U18 '[67.2828, 14.3711]'
  * lat       (lat) float64 67.28
  * lon       (lon) float64 14.37
  * alt       (alt) float64 70.0
Data variables:
    1001      (time, location) float64 14.5 14.5 14.3 14.4 ... 9.133 9.017 8.9
    3001      (time, location) float64 66.5 67.0 68.5 67.5 ... 77.07 77.68 78.3
    2001      (time, location) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    4002      (time, location) float64 4.7 5.0 5.3 5.6 7.0 ... 4.6 4.6 4.6 4.6
Attributes:
    weatherRessource:  Met Norway Locationforecast
    longitude:         [14.3711]
    latitude:          [67.2828]
    timeStart:         2021-05-17T09:00:00.000000000
    timeEnd:           2021-05-26T18:00:00.000000000
    parameters:        ['1001', '3001', '2001', '4002']

In [36]:
ds=norway.data(latitude=[67.2828,61.27], longitude=[14.3711,25.52], altitude=[70,0.],format='ds')
ds

<xarray.Dataset>
Dimensions:   (alt: 2, lat: 2, location: 2, lon: 2, time: 226)
Coordinates:
  * alt       (alt) float64 0.0 70.0
  * lat       (lat) float64 61.27 67.28
  * location  (location) <U18 '[61.27, 25.52]' '[67.2828, 14.3711]'
  * time      (time) datetime64[ns] 2021-05-17T09:00:00 ... 2021-05-26T18:00:00
  * lon       (lon) float64 14.37 25.52
Data variables:
    1001      (lon, lat, alt, time, location) float64 nan nan nan ... nan nan
    3001      (lon, lat, alt, time, location) float64 nan nan nan ... nan nan
    2001      (lon, lat, alt, time, location) float64 nan nan nan ... nan nan
    4002      (lon, lat, alt, time, location) float64 nan nan nan ... nan nan
Attributes:
    weatherRessource:  Met Norway Locationforecast
    longitude:         [14.3711, 25.52]
    latitude:          [61.27, 67.2828]
    timeStart:         2021-05-17T09:00:00.000000000
    timeEnd:           2021-05-26T18:00:00.000000000
    parameters:        ['1001', '3001', '2001', '4002']

**From ds you can see data as dataframe with function to_dataframe and exclude na value with dropna**
for more information on the http://xarray.pydata.org/en/stable/index.html

In [39]:
ds.to_dataframe().dropna()

1001  \
alt  lat     location           lon     time                             
0.0  61.2700 [61.27, 25.52]     25.5200 2021-05-17 09:00:00  14.700000   
                                        2021-05-17 10:00:00  15.400000   
                                        2021-05-17 11:00:00  15.900000   
                                        2021-05-17 12:00:00  16.200000   
                                        2021-05-17 13:00:00  16.600000   
...                                                                ...   
70.0 67.2828 [67.2828, 14.3711] 14.3711 2021-05-26 14:00:00   9.366667   
                                        2021-05-26 15:00:00   9.250000   
                                        2021-05-26 16:00:00   9.133333   
                                        2021-05-26 17:00:00   9.016667   
                                        2021-05-26 18:00:00   8.900000   

                                                                  3001  2001  \
alt  lat     location           lon     time                                   
0.0  61.2700 [61.27, 25.52]     25.5200 2021-05-17 09:00:00  69.900000   0.0   
                                        2021-05-17 10:00:00  68.400000   0.0   
                                        2021-05-17 11:00:00  68.700000   0.0   
                                        2021-05-17 12:00:00  65.800000   0.0   
                                        2021-05-17 13:00:00  64.700000   0.0   
...                                                                ...   ...   
70.0 67.2828 [67.2828, 14.3711] 14.3711 2021-05-26 14:00:00  75.833333   0.0   
                                        2021-05-26 15:00:00  76.450000   0.0   
                                        2021-05-26 16:00:00  77.066667   0.0   
                                        2021-05-26 17:00:00  77.683333   0.0   
                                        2021-05-26 18:00:00  78.300000   0.0   

                                                             4002  
alt  lat     location           lon     time                       
0.0  61.2700 [61.27, 25.52]     25.5200 2021-05-17 09:00:00   7.1  
                                        2021-05-17 10:00:00   8.1  
                                        2021-05-17 11:00:00   7.5  
                                        2021-05-17 12:00:00   8.1  
                                        2021-05-17 13:00:00   7.7  
...                                                           ...  
70.0 67.2828 [67.2828, 14.3711] 14.3711 2021-05-26 14:00:00   4.6  
                                        2021-05-26 15:00:00   4.6  
                                        2021-05-26 16:00:00   4.6  
                                        2021-05-26 17:00:00   4.6  
                                        2021-05-26 18:00:00   4.6  

[452 rows x 4 columns]